In [1]:
from neo4j import GraphDatabase
host = 'bolt://localhost:7687'
user = 'neo4j'
password = 'password'
driver = GraphDatabase.driver(host,auth=(user, password))

In [2]:
def run_query(query):
    with driver.session() as session:
        session.run(query)

In [3]:
from IPython.display import IFrame, HTML
import json
import uuid


def generate_vis(host, user, password, cypher, labels_json, relationships_json):
    html = """\
<html>
<head>
    <title>Neovis.js Simple Example</title>
            <style type="text/css">
                html, body {{
                    font: 16pt arial;
                }}
                #viz {{
                    width: 600px;
                    height: 800px;
                    font: 22pt arial;
                }}
            </style>
            <script src="https://cdn.neo4jlabs.com/neovis.js/v1.1.0/neovis.js"></script>
            <script
                    src="https://code.jquery.com/jquery-3.2.1.min.js"
                    integrity="sha256-hwg4gsxgFZhOsEEamdOYGBf13FyQuiTwlAQgxVSNgt4="
                    crossorigin="anonymous"></script>
            <script type="text/javascript">
                var viz;
                function draw() {{
                    var config = {{
                        container_id: "viz",
                        server_url: "{host}",
                        server_user: "{user}",
                        server_password: "{password}",
                        labels: {labels},
                        relationships: {relationships},
                        initial_cypher: "{cypher}"
                    }};
                    viz = new NeoVis.default(config);
                    viz.render();                    
                    viz.onVisualizationRendered(function(ctx) {{
                        let imageSrc = document.getElementsByTagName("canvas")[0].toDataURL();
                        console.log(imageSrc);
                        document.getElementById("viz-image").src=imageSrc;
                        //document.getElementById("viz").style="display:none";
                        
                        let kernel = IPython.notebook.kernel;
                        //let command = 'display(HTML('<img id="viz-image" width="300px" src="' + imageSrc + '" />';
                        let command = "foo = 'bar'";
                        kernel.execute(command);
                        
                    }});
                }}
            </script>
         </head>
        <body onload="draw()">
            <div id="viz"></div>
        </body>
    </html>
    """

    html = html.format(
        host=host,
        user=user,
        password=password,
        cypher=cypher,
        labels = json.dumps(labels_json),
        relationships=json.dumps(relationships_json)
    )

    unique_id = str(uuid.uuid4())
    filename = "graph-{}.html".format(unique_id)

    with open(filename, "w") as f:
        f.write(html)
    return IFrame(src=filename, width=1000, height=800)


In [4]:
def visualize_level(level, community):
    # Define cypher query
    if level > 1:
        cypher = """MATCH (p1:Person)-[r:INTERACTS_{rel_level}|:INTERACTS_{prev_level}]-(p2:Person) \
                    WHERE p1.community_{level} = {community} RETURN *""".format(
            rel_level=level if level != 4 else 45,level=level, prev_level=level -1, community=community)
    else:
        cypher = """MATCH (p1:Person)-[r:INTERACTS_{level}]-(p2:Person) \
                    WHERE p1.community_{level} = {community} RETURN *""".format(level=level, community=community)
    print(cypher)
    # Define relationships_json
    relationships_json = dict()
    for l in [level-1,level]:
        relationships_json["INTERACTS_{}".format(l if l != 4 else 45)] = {
                "caption": False
            }
    # Define labels_json    
    labels_json = {
        "Person": {
            "caption": "id",
            "community": "community_{}".format(level)
        }
    }

    return generate_vis(host, user, password, cypher, labels_json, relationships_json)

# Import

In [5]:
constraint_query = """CREATE CONSTRAINT ON (p:Person) ASSERT p.id IS UNIQUE;"""
run_query(constraint_query)

In [6]:
# https://networkofthrones.wordpress.com/
import_networks = """

UNWIND ['1','2','3','45'] as book
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-book' + book + '-edges.csv' as value
MERGE (source:Person{id:value.Source})
MERGE (target:Person{id:value.Target})
WITH source,target,value.weight as weight,book
CALL apoc.merge.relationship(source,'INTERACTS_' + book, {}, {weight:toFloat(weight)}, target) YIELD rel
RETURN distinct 'done'

"""
run_query(import_networks)

In [7]:
# https://www.kaggle.com/mylesoneill/game-of-thrones
import_deaths = """

LOAD CSV WITH HEADERS FROM "file:///character-deaths.csv" as row
WITH row WHERE row.`Book of Death` IS NOT NULL
MATCH (p:Person)
WHERE p.id = replace(row.Name,' ','-')
SET p.death_book = toInteger(row.`Book of Death`)

"""
run_query(import_deaths)

# Book 1

In [8]:
louvain_book1 = """

CALL algo.louvain('MATCH (p:Person)
WHERE (p)-[:INTERACTS_1]-()
RETURN id(p) as id'
,
'MATCH (p:Person)-[:INTERACTS_1]-(p1:Person)
RETURN id(p) as source, id(p1) as target'
,{graph:'cypher',direction:'BOTH',writeProperty:'community_1'})

"""
run_query(louvain_book1)

In [9]:
# Get Daenerys' community id 
get_daenerys_community_query = """
MATCH (p:Person{id:'Daenerys-Targaryen'})
RETURN p.community_1 as community
"""
with driver.session() as session:
    r = session.run(get_daenerys_community_query)

daenerys_community = list(r)[0]['community']
print(daenerys_community)

3


In [10]:
visualize_level(level=1,community=daenerys_community)

MATCH (p1:Person)-[r:INTERACTS_1]-(p2:Person)                     WHERE p1.community_1 = 3 RETURN *


# Book 2

In [11]:
louvain_book2 = """

CALL algo.louvain('MATCH (s:Person) WITH max(s.community_1) as max
                   MATCH (p:Person)
                   WHERE (p)-[:INTERACTS_1|:INTERACTS_2]-() AND NOT coalesce(p.death_book,10) < 2
                   RETURN id(p) as id,coalesce(p.community_1, max + id(p)) as community_1'
                   ,
                   'MATCH (p:Person)-[:INTERACTS_1|:INTERACTS_2]-(p1:Person)
                    RETURN id(p) as source, id(p1) as target'
                   ,{graph:'cypher',direction:'BOTH',writeProperty:'community_2',seedProperty:'community_1'})

"""
run_query(louvain_book2)

In [12]:
visualize_level(level=2,community=daenerys_community)

MATCH (p1:Person)-[r:INTERACTS_2|:INTERACTS_1]-(p2:Person)                     WHERE p1.community_2 = 3 RETURN *


# Book 3

In [13]:
louvain_book3 = """

CALL algo.louvain('MATCH (s:Person) WITH max(s.community_2) as max
                   MATCH (p:Person)
                   WHERE (p)-[:INTERACTS_2|:INTERACTS_3]-() AND NOT coalesce(p.death_book,10) < 3
                   RETURN id(p) as id,coalesce(p.community_2, max + id(p)) as community_2'
                    ,
                    'MATCH (p:Person)-[:INTERACTS_2|:INTERACTS_3]-(p1:Person)
                    RETURN id(p) as source, id(p1) as target'
                    ,{graph:'cypher',direction:'BOTH',writeProperty:'community_3',seedProperty:'community_2'})

"""
run_query(louvain_book3)

In [14]:
visualize_level(level=3,community=daenerys_community)

MATCH (p1:Person)-[r:INTERACTS_3|:INTERACTS_2]-(p2:Person)                     WHERE p1.community_3 = 3 RETURN *


# Book 4

In [15]:
louvain_book4 = """

CALL algo.louvain('MATCH (s:Person) WITH max(s.community_3) as max
MATCH (p:Person)
WHERE (p)-[:INTERACTS_3|:INTERACTS_45]-() AND NOT coalesce(p.death_book,10) < 4
RETURN id(p) as id,coalesce(p.community_3, max + id(p)) as community_3'
,
'MATCH (p:Person)-[:INTERACTS_3|:INTERACTS_45]-(p1:Person)
RETURN id(p) as source, id(p1) as target'
,{graph:'cypher',direction:'BOTH',writeProperty:'community_4',seedProperty:'community_3'})

"""
run_query(louvain_book4)

In [16]:
visualize_level(level=4,community=daenerys_community)

MATCH (p1:Person)-[r:INTERACTS_45|:INTERACTS_3]-(p2:Person)                     WHERE p1.community_4 = 3 RETURN *
